Pip Install

In [1]:
!pip install langdetect
!pip install bert-score
!pip install transformers
!pip install kagglehub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=c401ec24b768a1915650295f5e6786b138ee0b3868e4e5dfd1891deaec92c7bd
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

### Data Import

In [2]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("beridzeg45/book-reviews")

print("Path to dataset files:", path)
df_books = pd.read_csv(path + "/Book Reviews.csv")
df_books.head()

100%|██████████| 28.0M/28.0M [00:01<00:00, 22.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/beridzeg45/book-reviews/versions/1


,Unnamed: 0,Book,Review,Review Date
0,0,To Kill a Mockingbird,/// gentle reminder that this is not the time ...,"March 24, 2022"
1,1,To Kill a Mockingbird,\n|\n|6.0 stars. I know I am risking a serious...,"May 24, 2011"
2,2,To Kill a Mockingbird,\n|\n|Looking for a new book but don't want to...,"December 10, 2020"
3,3,To Kill a Mockingbird,"To Kill a Mockingbird, Harper Lee|To Kill a Mo...","July 1, 2022"
4,4,To Kill a Mockingbird,Why is it when I pick up | To Kill A Mockingbi...,"October 25, 2009"


### Inspect Data

In [3]:
df_books.head()

,Unnamed: 0,Book,Review,Review Date
0,0,To Kill a Mockingbird,/// gentle reminder that this is not the time ...,"March 24, 2022"
1,1,To Kill a Mockingbird,\n|\n|6.0 stars. I know I am risking a serious...,"May 24, 2011"
2,2,To Kill a Mockingbird,\n|\n|Looking for a new book but don't want to...,"December 10, 2020"
3,3,To Kill a Mockingbird,"To Kill a Mockingbird, Harper Lee|To Kill a Mo...","July 1, 2022"
4,4,To Kill a Mockingbird,Why is it when I pick up | To Kill A Mockingbi...,"October 25, 2009"


In [4]:
print("\nMissing values per column:\n", df_books.isnull().sum())


Missing values per column:
 Unnamed: 0       0
Book             0
Review         309
Review Date      0
dtype: int64


In [5]:
#Keep only English Review Text

from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Set seed for consistency
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"
# Only detect language if not empty
df_books['language'] = df_books['Review'].apply(lambda x: detect_language(str(x)) if str(x).strip() else "unknown")

# Filter for English-only reviews
df_books = df_books[df_books['language'] == 'en']


In [6]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23057 entries, 0 to 32077
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   23057 non-null  int64 
 1   Book         23057 non-null  object
 2   Review       23057 non-null  object
 3   Review Date  23057 non-null  object
 4   language     23057 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.1+ MB


In [7]:
#Merge Reviews to one row review per book
df_books['Review'] = df_books['Review'].astype(str)
df_reviews = df_books.groupby('Book')['Review'].apply(lambda reviews: ' '.join(reviews)).reset_index()
df_reviews.columns = ['Book', 'Review']
df_reviews.head()


<ipython-input-7-fce335649d3f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Review'] = df_books['Review'].astype(str)


,Book,Review
0,10:04,I’m going to let the text of 10:04 by Ben Lern...
1,1984,YOU. ARE. THE. DEAD.| Oh my God. I got the chi...
2,"1Q84 (1Q84, #1-3)",1Q84 is undoubtedly the biggest literary let-d...
3,2001: A Space Odyssey,"The book is always better than the film, but I..."
4,2666,Roberto Bolaño's |2666| has been described as ...


In [8]:
df_reviews[df_reviews.duplicated()]


,Book,Review


### Data Pre Processing

In [9]:
#Clean text for PEGASUS input

def clean_text(text):
    text = text.replace('\n', ' ').replace('\r', ' ').strip()
    return text

df_reviews['Review'] = df_reviews['Review'].apply(clean_text)

df_reviews.head()


,Book,Review
0,10:04,I’m going to let the text of 10:04 by Ben Lern...
1,1984,YOU. ARE. THE. DEAD.| Oh my God. I got the chi...
2,"1Q84 (1Q84, #1-3)",1Q84 is undoubtedly the biggest literary let-d...
3,2001: A Space Odyssey,"The book is always better than the film, but I..."
4,2666,Roberto Bolaño's |2666| has been described as ...


### PEGASUS Class

In [10]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch

In [49]:
#Generation Tuner PEGASUS Class

class PegasusSummarizer:
    def __init__(self, model_name="google/pegasus-xsum", device=None, generation_kwargs=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = PegasusTokenizer.from_pretrained(model_name)
        self.model = PegasusForConditionalGeneration.from_pretrained(model_name).to(self.device)
        self.generation_kwargs = generation_kwargs or { #initalise summary generation tuners
            "max_length": 150, #Maximum length of the summary in tokens
            "min_length": 50, #Minimum length of the summary in tokens
            "do_sample": True, #Enables sampling instead of greedy decoding (for creativity)
            "top_k": 50, #Sample only from the top 50 most probable next words
            "top_p": 0.95, #Nucleus sampling: consider tokens with 95% cumulative probability
            "temperature": 0.8, #Controls randomness: <1 = more focused, >1 = more random
            "repetition_penalty": 1.2 #Penalizes repeated phrases or words to avoid looping
}


    def summarize(self, text, max_input_length=512):
        if not isinstance(text, str) or not text.strip():
            return ""

        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding="longest",
            max_length=max_input_length
        ).to(self.device)

        summary_ids = self.model.generate(
            inputs["input_ids"],
            **self.generation_kwargs
        )

        return self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    def summarize_books(self, df, text_column="Review", title_column="Book"):
        results = []
        for idx, row in df.iterrows():
            Book = row[title_column]
            Reviews = row[text_column]
            Summary = self.summarize(Reviews)
            results.append({
                "book_number": f"Book {idx + 1}",
                "Book": Book,
                "Summary": Summary
            })
            print(f"\n Book {idx + 1}: {Book}\n Summary: {Summary}\n{'-'*80}")
        return pd.DataFrame(results)




In [50]:
#Selected books for summary test
lBooks = ['To Kill a Mockingbird', '1984', 'Jane Eyre', 'Animal Farm',
          'Crime and Punishment', 'The Catcher in the Rye', 'Cataract',
          "I'm Not Scared", 'Reasons to Live', 'Night Boat to Tangier',
          'Infinite Jest']


# Filter only for selected books
filtered_books_df = df_reviews[df_reviews['Book'].isin(lBooks)]

# Group reviews per book (if not already done)
df_selected = filtered_books_df.groupby('Book')['Review'].apply(lambda x: ' '.join(x)).reset_index()
df_selected.columns = ['Book', 'Review']


In [51]:
#Common creative decoding parameters (balanced sampling) variation I

custom_params = {
    "max_length": 200,
    "min_length": 60,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "temperature": 0.8,
    "repetition_penalty": 1.2
}

summarizer = PegasusSummarizer(generation_kwargs=custom_params)
summary_df = summarizer.summarize_books(df_selected)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 Book 1: 1984
 Summary: In our series of letters from African journalists, film-maker and columnist Farai Sevenzo reviews George Orwell's dystopian novel 1984, which tells the story of Winston, a young man who is sent to live in a society where "reading and writing could one day be outlawed." by Farai Sevenzo
--------------------------------------------------------------------------------

 Book 2: Animal Farm
 Summary: George Orwell's Animal Farm was pulled from sale by Amazon in 2009 after the company discovered that it did not have the rights to publish the book on its Kindle e-book reader, but the company has since said it "will not remove books from customers' devices in these circumstances". After the move prompted outcry and comparisons to Nineteen Eighty-Four itself, Amazon spokesman Drew Herdener stated that the company is "changing our systems so that in the future we will not remove books from customers' devices in these circumstances."
-------------------------------------

In [52]:
#Deterministic beam search (no randomness) varaiation II
custom_params = {
    "max_length": 150,
    "min_length": 60,
    "do_sample": False, #Set to false = no randomness
    "num_beams": 5, #Deterministic beam search over 5 paths
    "repetition_penalty": 1.2
}

summarizer = PegasusSummarizer(generation_kwargs=custom_params)
summary_finetuneII = summarizer.summarize_books(df_selected)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 Book 1: 1984
 Summary: In our series of letters from African journalists, film-maker and columnist Sharmila Tagore reflects on her experience of reading George Orwell's dystopian novel 1984, which she describes as "the best book I've read in a very long time" and "the best thing I've read in a very long time."
--------------------------------------------------------------------------------

 Book 2: Animal Farm
 Summary: George Orwell's Animal Farm is one of the most banned books in the world, and Amazon pulled it from sale in 2009 after discovering that the publisher lacked rights to publish the titles in question, prompting comparisons to Orwell's novel Nineteen Eighty-Four, which was also banned in many countries.
--------------------------------------------------------------------------------

 Book 3: Cataract
 Summary: Mykhaylo Osadchy's memoir, "Cataract", about his imprisonment by the KGB in the mid-sixties, has been translated into English by Marco Caynnyk, editor of the Ukr

In [26]:
#High creativity and more expresiveness variation III
custom_params = {
    "max_length": 220,
    "min_length": 80,
    "do_sample": True,
    "top_k": 40,
    "top_p": 0.90,
    "temperature": 1.0,
    "repetition_penalty": 1.0
}
summarizer = PegasusSummarizer(generation_kwargs=custom_params)
summary_finetuneIII = summarizer.summarize_books(df_selected)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 Book 1: 1984
 Summary: In our series of letters from African journalists, film-maker and columnist Sharmila Tagore reflects on her experience of reading George Orwell's dystopian novel 1984, which tells the story of Winston Churchill's rise to power in a society ruled by a party known as the Party (War is Peace), and how she relates it to her own experiences as a writer and a woman in today's world.
--------------------------------------------------------------------------------

 Book 2: Animal Farm
 Summary: George Orwell's Animal Farm is one of the most banned books in the world, and Amazon pulled it from sale in 2009 after discovering that the publisher lacked rights to publish the title in question, sparking outrage and comparisons to Orwell's novel Nineteen Eighty-Four, which was banned in the US after it was found to be "offensive to women" and "insulting" to Muslims.
--------------------------------------------------------------------------------

 Book 3: Cataract
 Summary: 

Summarize selected test books:

#### Full Book Summary

*only run and remove hashtags if needed, runtime 5hr+*

In [14]:

# Run summarization for all books with standard class generation_kwargs

#summarizer = PegasusSummarizer(generation_kwargs=custom_params)
#full_summary = summarizer.summarize_books(df_selected)



### BERT Score Evaluation

### Mean BERT Score over 10 sample books

In [54]:
from bert_score import score

# Get summary and source columns from summary_df and df_selected
summaries = summary_df['Summary'].tolist()
originals = df_selected['Review'].tolist()


# Compute BERTScore
P, R, F1 = score(summaries, originals, lang="en")

print(f"Precision: {P.mean():.4f}")
print(f"Recall: {R.mean():.4f}")
print(f"F1 Score: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8580
Recall: 0.7887
F1 Score: 0.8218


In [55]:
from bert_score import score

# Get summary and source columns from summary_finetuneII and df_selected
summaries = summary_finetuneII['Summary'].tolist()
originals = df_selected['Review'].tolist()


# Compute BERTScore
P, R, F1 = score(summaries, originals, lang="en")

print(f"Precision: {P.mean():.4f}")
print(f"Recall: {R.mean():.4f}")
print(f"F1 Score: {F1.mean():.4f}")



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8452
Recall: 0.7884
F1 Score: 0.8153


In [56]:
from bert_score import score

# Get summary and source columns from summary_finetuneIII and df_selected
summaries = summary_finetuneIII['Summary'].tolist()
originals = df_selected['Review'].tolist()


# Compute BERTScore
P, R, F1 = score(summaries, originals, lang="en")

print(f"Precision: {P.mean():.4f}")
print(f"Recall: {R.mean():.4f}")
print(f"F1 Score: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8439
Recall: 0.7897
F1 Score: 0.8158


### BERT Score per Book

In [57]:

# Match the 10 books and summaries for variation I
originals = df_selected['Review'].tolist()
summaries = summary_df['Summary'].tolist()

# Compute BERTScores for each book
P, R, F1 = score(summaries, originals, lang="en")

# Store them in your summary_df
summary_df['bert_precision'] = P.tolist()
summary_df['bert_recall'] = R.tolist()
summary_df['bert_f1'] = F1.tolist()

# Check result
print(summary_df[['Book', 'bert_f1']].head())


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                   Book   bert_f1
0                  1984  0.802848
1           Animal Farm  0.851377
2              Cataract  0.859046
3  Crime and Punishment  0.822370
4        I'm Not Scared  0.856575


In [58]:
# Match the 10 books and summaries for variation II
originals = df_selected['Review'].tolist()
summaries = summary_finetuneII['Summary'].tolist()

# Compute BERTScores for each book
P, R, F1 = score(summaries, originals, lang="en")

# Store them in your summary_df
summary_finetuneII['bert_precision'] = P.tolist()
summary_finetuneII['bert_recall'] = R.tolist()
summary_finetuneII['bert_f1'] = F1.tolist()

# Check result
print(summary_finetuneII[['Book', 'bert_f1']].head())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                   Book   bert_f1
0                  1984  0.800437
1           Animal Farm  0.837467
2              Cataract  0.859019
3  Crime and Punishment  0.809803
4        I'm Not Scared  0.843911


In [59]:
# Match the 10 books and summaries for variation III
originals = df_selected['Review'].tolist()
summaries = summary_finetuneIII['Summary'].tolist()

# Compute BERTScores for each book
P, R, F1 = score(summaries, originals, lang="en")

# Store them in your summary_df
summary_finetuneIII['bert_precision'] = P.tolist()
summary_finetuneIII['bert_recall'] = R.tolist()
summary_finetuneIII['bert_f1'] = F1.tolist()

# Check result
print(summary_finetuneIII[['Book', 'bert_f1']].head())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                   Book   bert_f1
0                  1984  0.808190
1           Animal Farm  0.831610
2              Cataract  0.823848
3  Crime and Punishment  0.826520
4        I'm Not Scared  0.852111


### Manual Evaluation

Orginal vs Generated Review

In [60]:
#Variation I

sample = summary_df

for _, row in sample.iterrows():
    book_title = row['Book']
    summary = row['Summary']
    original = df_reviews[df_reviews['Book'] == book_title]['Review'].values[0]

    print(f"\n {book_title}")
    print(f" Summary: {summary}")
    print(f" Original Excerpt: {original[:300]}...\n")



 1984
 Summary: In our series of letters from African journalists, film-maker and columnist Farai Sevenzo reviews George Orwell's dystopian novel 1984, which tells the story of Winston, a young man who is sent to live in a society where "reading and writing could one day be outlawed." by Farai Sevenzo
 Original Excerpt: YOU. ARE. THE. DEAD.| Oh my God. I got the chills so many times toward the end of this book. It completely blew my mind. It managed to surpass my high expectations AND be nothing at all like I expected. Or in Newspeak "Double Plus Good."|Let me preface this with an apology. If I sound stunningly ina...


 Animal Farm
 Summary: George Orwell's Animal Farm was pulled from sale by Amazon in 2009 after the company discovered that it did not have the rights to publish the book on its Kindle e-book reader, but the company has since said it "will not remove books from customers' devices in these circumstances". After the move prompted outcry and comparisons to Nineteen Eighty

In [61]:
#Variation II
sample = summary_finetuneII

for _, row in sample.iterrows():
    book_title = row['Book']
    summary = row['Summary']
    original = df_reviews[df_reviews['Book'] == book_title]['Review'].values[0]

    print(f"\n {book_title}")
    print(f" Summary: {summary}")
    print(f" Original Excerpt: {original[:300]}...\n")


 1984
 Summary: In our series of letters from African journalists, film-maker and columnist Sharmila Tagore reflects on her experience of reading George Orwell's dystopian novel 1984, which she describes as "the best book I've read in a very long time" and "the best thing I've read in a very long time."
 Original Excerpt: YOU. ARE. THE. DEAD.| Oh my God. I got the chills so many times toward the end of this book. It completely blew my mind. It managed to surpass my high expectations AND be nothing at all like I expected. Or in Newspeak "Double Plus Good."|Let me preface this with an apology. If I sound stunningly ina...


 Animal Farm
 Summary: George Orwell's Animal Farm is one of the most banned books in the world, and Amazon pulled it from sale in 2009 after discovering that the publisher lacked rights to publish the titles in question, prompting comparisons to Orwell's novel Nineteen Eighty-Four, which was also banned in many countries.
 Original Excerpt: Amazon's very Orwellian i

In [62]:
#Variation III
sample = summary_finetuneIII

for _, row in sample.iterrows():
    book_title = row['Book']
    summary = row['Summary']
    original = df_reviews[df_reviews['Book'] == book_title]['Review'].values[0]

    print(f"\n {book_title}")
    print(f" Summary: {summary}")
    print(f" Original Excerpt: {original[:300]}...\n")


 1984
 Summary: In our series of letters from African journalists, film-maker and columnist Sharmila Tagore reflects on her experience of reading George Orwell's dystopian novel 1984, which tells the story of Winston Churchill's rise to power in a society ruled by a party known as the Party (War is Peace), and how she relates it to her own experiences as a writer and a woman in today's world.
 Original Excerpt: YOU. ARE. THE. DEAD.| Oh my God. I got the chills so many times toward the end of this book. It completely blew my mind. It managed to surpass my high expectations AND be nothing at all like I expected. Or in Newspeak "Double Plus Good."|Let me preface this with an apology. If I sound stunningly ina...


 Animal Farm
 Summary: George Orwell's Animal Farm is one of the most banned books in the world, and Amazon pulled it from sale in 2009 after discovering that the publisher lacked rights to publish the title in question, sparking outrage and comparisons to Orwell's novel Ninete

### Export Results from different models

In [63]:
#Variation I

# Get matching originals for the summaries
originals = df_selected['Review'].tolist()

# Add original text to summary_df
summary_df['original'] = originals

# Reorder and select columns for clarity
export_df = summary_df[[
    'Book', 'original', 'Summary', 'bert_precision', 'bert_recall', 'bert_f1'
]]

# Save to CSV
export_df.to_csv("pegasus_summary_VI.csv", index=False)

In [64]:
#Variation II
# Get matching originals for the summaries
originals = df_selected['Review'].tolist()

# Add original text to summary_df
summary_finetuneII['original'] = originals

# Reorder and select columns for clarity
export_df = summary_df[[
    'Book', 'original', 'Summary', 'bert_precision', 'bert_recall', 'bert_f1'
]]

# Save to CSV
export_df.to_csv("pegasus_summary_VII.csv", index=False)


In [65]:
#Variation III
# Get matching originals for the summaries
originals = df_selected['Review'].tolist()

# Add original text to summary_df
summary_finetuneIII['original'] = originals

# Reorder and select columns for clarity
export_df = summary_finetuneIII[[
    'Book', 'original', 'Summary', 'bert_precision', 'bert_recall', 'bert_f1'
]]

# Save to CSV
export_df.to_csv("pegasus_summary_VIII.csv", index=False)